In [ ]:
import tensorflow as tf
import os

In [1]:
import gc
gc.collect()

0

In [3]:
# DONT RUN THIS CELL!!

base_dir = r'C:\Tensorflow\Animals'
base_dir

train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir, 'test')

# MAKE SUBFOLDER FOR TRAINING IMAGE
os.makedirs(os.path.join(train_dir,'cats'),exist_ok=True)
os.makedirs(os.path.join(train_dir,'dogs'),exist_ok=True)
os.makedirs(os.path.join(train_dir,'snakes'),exist_ok=True)

# MAKE SUBFOLDER FOR TESTING IMAGE
os.makedirs(os.path.join(test_dir,'cats'),exist_ok=True)
os.makedirs(os.path.join(test_dir,'dogs'),exist_ok=True)
os.makedirs(os.path.join(test_dir,'snakes'),exist_ok=True)

In [ ]:
# THIS CELL IS TO SPLIT IMAGE INTO 2 SUBFOLDER (TRAIN AND TEST)
# ANYWAY , DONT RUN THIS CELL!!

from sklearn.model_selection import train_test_split
import shutil

animals = ['cats','dogs','snakes']

for animal in animals: # MOVE FILE FOR EACH ANIMAL CLASS
    animals_path = os.path.join(base_dir,animal) # PATH FOR EVERY SUBFOLDER CATEGORY
    scan_all_files = os.listdir(animals_path)  #SCAN AND GET IMAGE IN ANIMAL FOLDER

    #SPLIT ANIMAL FILES INTO TRAIN AND TEST
    train_files , test_files = train_test_split(scan_all_files,test_size=0.2,random_state=21,shuffle=True) 

    for filename in train_files: 
        #COPY EACH FILE FROM REAL ANIMAL FOLDER TO TRAIN ANIMAL FOLDER
        shutil.copy(os.path.join(animals_path,filename),os.path.join(train_dir,animal)) #COPY EACH FILE FROM REAL ANIMAL FOLDER TO TRAIN ANIMAL FOLDER

    for filename in test_files:
        #COPY EACH FILE FROM ANIMAL FOLDER TO TEST ANIMAL FOLDER
        shutil.copy(os.path.join(animals_path,filename),os.path.join(test_dir,animal))
    



In [ ]:
# THIS CELL IS TO SEE HOW MUCH SIZE OF THE IMAGE
# U CAN RUN THIS CELL 

from PIL import Image
import os

# ACCESSED IMAGE PATH
directory = r'..\Animals\train\snakes'  # Contoh path

# TO SEE SIZE OF IMAGE
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):  
        img_path = os.path.join(directory, filename)
        with Image.open(img_path) as img:
            print(f'{filename}: {img.size}') 
            break                            # JUST WANNA SEE SIZE OF 1 IMAGE


In [3]:
# THIS CELL IS TO IMPORT IMAGE FILE AT LOCAL PC

import tensorflow as tf

# IMPORT TRAIN IMAGE FROM ANIMALS FOLDER
train_folder = tf.keras.utils.image_dataset_from_directory(
    directory='../Animals/train',
    labels='inferred',
    batch_size=10,
    image_size=(180,180),
    label_mode='categorical'
)

# IMPORT TEST IMAGE FROM ANIMALS FOLDER
test_folder = tf.keras.utils.image_dataset_from_directory(
    directory='../Animals/test',
    labels='inferred',
    batch_size=10,
    image_size=(180,180),
    label_mode='categorical'
)

Found 2400 files belonging to 3 classes.
Found 600 files belonging to 3 classes.


In [4]:
# USE PREFETCH FOR FASTER TRAINING
train_folder = train_folder.prefetch(buffer_size=tf.data.AUTOTUNE).cache().shuffle(2000) # FOR TRAINING DATASET
test_folder = test_folder.prefetch(buffer_size=tf.data.AUTOTUNE).cache().shuffle(600)

train_folder

<ShuffleDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

In [5]:
# CREATING CONVOLUTION ARCHITECTURE

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(180,180,3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),

    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),

    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=3,activation='softmax')
],name='CNN-Model'
)

In [6]:
model.summary()

Model: "CNN-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 128)       73

In [6]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],run_eagerly=False)

In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        
        if logs is not None and 'val_accuracy' in logs:
            if logs['val_accuracy'] >= 0.97:
                print('\nTraining Successfully')
                self.model.stop_training = True

In [8]:

history = model.fit(train_folder,epochs=15,verbose=1,validation_data=[test_folder],batch_size=1)

Epoch 1/15
240/240 [==============================] - 12s 31ms/step - loss: 0.9946 - accuracy: 0.4842 - val_loss: 0.7910 - val_accuracy: 0.6467
Epoch 2/15
240/240 [==============================] - 7s 29ms/step - loss: 0.8181 - accuracy: 0.6062 - val_loss: 0.7670 - val_accuracy: 0.6200
Epoch 3/15
240/240 [==============================] - 7s 28ms/step - loss: 0.7242 - accuracy: 0.6646 - val_loss: 0.7888 - val_accuracy: 0.6183
Epoch 4/15
240/240 [==============================] - 7s 27ms/step - loss: 0.6386 - accuracy: 0.7221 - val_loss: 0.6995 - val_accuracy: 0.6867
Epoch 5/15
240/240 [==============================] - 7s 27ms/step - loss: 0.5354 - accuracy: 0.7679 - val_loss: 0.7753 - val_accuracy: 0.6950
Epoch 6/15
240/240 [==============================] - 7s 27ms/step - loss: 0.4369 - accuracy: 0.8233 - val_loss: 0.7566 - val_accuracy: 0.6983
Epoch 7/15
240/240 [==============================] - 7s 28ms/step - loss: 0.3606 - accuracy: 0.8671 - val_loss: 0.8836 - val_accuracy: 0.676